In [101]:
# This is an implementation of imp step based on Gorte paper 

In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
from algorithms import mm_utils
from Fuzzy.FIS1 import FIS1


In [2]:
def get_bearing(point1, point2):
    # this code calculate the bearing of any given pair of longitude, latitude  
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(point1[0], point1[1], point2[0], point2[1])
    return fwd_azimuth

def edge_bearing(edge):
    # this function calculate the bearing from the starting and ending node of each road segment
    bearing = get_bearing(edge[0], edge[len(edge) - 1])
    return(bearing)
    
def conv_angle(angle):
    # this function convert angle from -pi,pi to 0,2*pi
    if angle < 0 :
        angle = angle + 360
    return(angle)

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans

def err_polygon(curr_loc, err_size):
    # function that output shapely polygon for point error bound
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y
    
    err_coord = [[x - err_size, y + err_size], 
                 [x + err_size, y + err_size],
                 [x + err_size, y - err_size],
                 [x - err_size, y - err_size]]

    poly_coord = Polygon(err_coord)
    # #print(ply_coord)
    df = {'Attribute' : ['name1'], 'geometry':poly_coord}

    #projected to UTM 31 
    err_poly = gpd.GeoDataFrame(df, geometry = 'geometry', crs = "EPSG:32631")
    
    return err_poly

# Project data into UTM 31 

In [3]:
# read data 
gdf = pd.read_pickle('envirocar.pkl')

# get road network 
# Get the bounding box
bbox = gdf.total_bounds

# 'total_bounds' returns a tuple with (minx, miny, maxx, maxy) values
minx, miny, maxx, maxy = bbox

# Download a map by specifying the bounding box
# and draw the graph
G = ox.graph.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive') 

# extract road info 
nodes, edges = ox.graph_to_gdfs(G)
# project edges into UTM 31 projection 
edges_utm = edges.to_crs({'init': 'epsg:32631'})

# append latitude and longitude to utm edges 
edges_utm['lat_lon'] = edges['geometry']

# convert osmid into unique string id 
edges_utm['str_id'] = edges_utm['osmid'].apply(conc)

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# Project trajectory into utm 31

In [4]:
# convert gdf to utm 31 projection 
# initialize projection to standard WGS 84 
gdf.crs = {'init': 'epsg:4326'}

# convert gdf to UTM31 projection  
gdf_utm = gdf.to_crs({'init': 'epsg:32631'})

# append latitude and longitude to utm edges 
gdf_utm['lat_lon'] = gdf['geometry']

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# Perform IMP 

In [29]:
# initialization for IMP
count = 0 
same_link = 0
stop_iter = False
iter = 0
err_size = 38

# saving answer for debugging purposes 
# edge_link saves all the candidate link name for each iteration 
# final answer is stored in the edge_link variable 
edge_link = []
# fis_res saves the output of FIS algorithm at every iteration 
fis_res = []
# HE_iter saves the heading error values for each candidate edge at any given iteration 
HE_iter = [] 
#curr_pos
curr_pos_list = []
# save candidate link name each iteration  
candidate_link_res = []


while stop_iter == False :
    # extract current location at given iteration 
    curr_loc = gdf_utm.iloc[iter].to_frame().T
    # save the iteration current position as a list
    curr_pos_list.append(curr_loc)
    
    #-----------------------------------------------------------------------
    # input should be location and error size 
    # create rectangular polygon 
    err_poly = err_polygon(curr_loc, err_size)

    # to plot error polygon for debugging
    # err_poly.plot()

    #---------------------------------------------------------------------------
    
    # Check for intersection and containment using geopandas
    intersects = gpd.sjoin(err_poly, edges_utm, op='intersects')
    contains = gpd.sjoin(err_poly, edges_utm, op='contains')

    if (len(intersects) + len(contains)) <=0:
        print(['no edeges intersects with error bound at iteration number', iter + 1])
    else:    
        # perform IMP only when there is edge intersects with error bound
        print(['edges found at iteration number', iter + 1])

        # extract index from edges that intersect with error polygon 
        int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
        # extract index from edges that contained in the error polygon 
        cont_index = contains[['index_right0', 'index_right1', 'index_right2']]

        # merge index
        index = pd.concat([int_index, cont_index])
        # drop duplicate
        index = index.drop_duplicates()

        # initialize candidate edges 
        appended_edge = []

        # extract candidate eges  
        for i in range(len(index)):
            edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
            appended_edge.append(edge_list)

        candidate_link = edges_utm.loc[appended_edge]

        #save candidate link name 
        candidate_link_res.append(candidate_link['osmid'])

        # calculate perpendicular distance 
        # initialize list that hold perpendicular distance between points and edges
        p_dist = []

        # calculate perpendicular distance between current point and 
        for i in range(len(candidate_link)):
            p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])

        # attach perpendicular distance to candidate link 
        candidate_link["perp_dist"] = p_dist

        # print(candidate_link)

        # calculate heading error
        # convert lat lon into tupple coordinate 
        candidate_link['lat_lon_pair'] = candidate_link.lat_lon.apply(lambda geom: list(geom.coords))

        # calculate bearing frome start and end node for each candidate link (see notes below)
        bearing_raw = candidate_link['lat_lon_pair'].apply(edge_bearing)

        # convert bearing from -pi, pi to 0, 2pi range
        candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)

        # heading error = abs(gps heading - edge bearing)
        candidate_link['heading_error'] = abs(candidate_link['edge_heading'] - gdf['GPS Bearing'].iloc[iter])

        # initialize input for FIS
        PD = candidate_link['perp_dist']
        HE = candidate_link['heading_error']
        speed = np.repeat(gdf['GPS Speed'][iter], len(candidate_link))
        hdop = np.repeat(gdf['GPS HDOP'][iter], len(candidate_link))

        # save HE value every iter 
        HE_iter.append(HE)
        # rearrange new data to the input of fis1  
        new_data = np.array([speed, HE, PD, hdop]).T

        # calculating FIS
        pred =[]
        for i in range(len(new_data)):
            pred.append(FIS1(new_data[i,:], plot = False))

        # print(pred)
        # save fis result 
        fis_res.append(pred)

        # pick candidate link based on 
        index = pred.index(max(pred))

        edge_link.append(candidate_link['osmid'].iloc[index])

        # check if the current position and previous position is in the same edge
        if count > 0:
            if edge_link[count] == edge_link[count - 1]:
                same_link = same_link + 1
            else:
                same_link = 0

        # check to stop the for loop if three points belong to the same edge
        if same_link == 1:
            stop_iter = True
        else:
            count = count + 1

    #update iteration 
    iter = iter + 1     

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 1]
['no edeges intersects with error bound at iteration number', 2]
['no edeges intersects with error bound at iteration number', 3]
['no edeges intersects with error bound at iteration number', 4]
['no edeges intersects with error bound at iteration number', 5]
['no edeges intersects with error bound at iteration number', 6]
['no edeges intersects with error bound at iteration number', 7]
['no edeges intersects with error bound at iteration number', 8]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 9]
['no edeges intersects with error bound at iteration number', 10]
['no edeges intersects with error bound at iteration number', 11]
['edges found at iteration number', 12]
['edges found at iteration number', 13]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: Th

# Plot Result

In [28]:
%matplotlib tk
# This is how we  visualize edges and error bound 
#print(edges)
print(type(edges))

# find which edges is selected at time point
# find index of the edge id
loc = np.where(edges_utm["str_id"] == conc(edge_link[count]))

# find the last two position for IMP
poly_1 = err_polygon(curr_pos_list[count + iter - 2], err_size)
poly_2 = err_polygon(curr_pos_list[count + iter - 3], err_size)

#Save selected edge 
answer_loc = edges_utm.iloc[loc]

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
point_locs.iloc[0:iter, :].plot(ax = ax)

#current location versus edges
#curr_loc.plot(ax=ax)

#err coord 
# better if we just take location at the last and use error bound function 
poly_1.plot(ax=ax, facecolor="none")
poly_2.plot(ax=ax, facecolor="none")

# this plot all the road system 
edges_utm.plot(ax=ax)
# this plot the selected edge at time point 
answer_loc.plot(ax=ax, cmap = "Reds")

#print(intersects['index_right'])

<class 'geopandas.geodataframe.GeoDataFrame'>


<Axes: >

# Debugging 

In [146]:
iter = 14
# convert ot UTM zone 31 which is equal to epsg32631 according to https://spatialreference.org/ref/epsg/32631/
curr_loc = gdf_utm.iloc[iter].to_frame().T
curr_pos_list.append(curr_loc)
#-----------------------------------------------------------------------
# this part could be made into a function that create error polygon 
# input should be location and error size 
# create rectangular polygon 
err_poly = err_polygon(curr_loc, err_size)

# plot error polygon for debugging
# err_poly.plot()

#---------------------------------------------------------------------------

# Check for intersection and containment using geopandas
intersects = gpd.sjoin(err_poly, edges_utm, op='intersects')
contains = gpd.sjoin(err_poly, edges_utm, op='contains')
# skip iteration if no edges are detected 
if (len(intersects) + len(contains)) >0:
    print(['edges found at iteration number', iter + 1])

    # extract index from edges that intersect with error polygon 
    int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
    # extract index from edges that contained in the error polygon 
    cont_index = contains[['index_right0', 'index_right1', 'index_right2']]

    # merge index
    index = pd.concat([int_index, cont_index])
    # drop duplicate
    index = index.drop_duplicates()

    # initialize candidate edges 
    appended_edge = []

    # extract candidate eges  
    for i in range(len(index)):
        edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
        appended_edge.append(edge_list)

    candidate_link = edges_utm.loc[appended_edge]

    #save candidate link name 
    candidate_link_res.append(candidate_link['osmid'])

    # calculate perpendicular distance 
    # initialize list that hold perpendicular distance between points and edges
    p_dist = []

    # calculate perpendicular distance between current point and 
    for i in range(len(candidate_link)):
        p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])

    # attach perpendicular distance to candidate link 
    candidate_link["perp_dist"] = p_dist

    # print(candidate_link)

    # calculate heading error
    # convert lat lon into tupple coordinate 
    candidate_link['lat_lon_pair'] = candidate_link.lat_lon.apply(lambda geom: list(geom.coords))

    # calculate bearing frome start and end node for each candidate link (see notes below)
    bearing_raw = candidate_link['lat_lon_pair'].apply(edge_bearing)

    # convert bearing from -pi, pi to 0, 2pi range
    candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)

    # heading error = abs(gps heading - edge bearing)
    candidate_link['heading_error'] = abs(candidate_link['edge_heading'] - gdf['GPS Bearing'].iloc[iter])

    # initialize input for FIS
    PD = candidate_link['perp_dist']
    HE = candidate_link['heading_error']
    speed = np.repeat(gdf['GPS Speed'][iter], len(candidate_link))
    hdop = np.repeat(gdf['GPS HDOP'][iter], len(candidate_link))

    # save HE value every iter 
    HE_iter.append(HE)
    # rearrange new data to the input of fis1  
    new_data = np.array([speed, HE, PD, hdop]).T

    # calculating FIS
    pred =[]
    for i in range(len(new_data)):
        pred.append(FIS1(new_data[i,:], plot = False))

    # print(pred)
    # save fis result 
    fis_res.append(pred)

    # pick candidate link based on 
    index = pred.index(max(pred))

    edge_link.append(candidate_link['osmid'].iloc[index])

    # check if the current position and previous position is in the same edge
    if count > 0:
        if edge_link[count] == edge_link[count - 1]:
            same_link = same_link + 1
        else:
            same_link = 0

    # check to stop the for loop if three points belong to the same edge
    if same_link == 1:
        stop_iter = True
    else:
        count = count + 1

#update iteration 
iter = iter + 1 


['edges found at iteration number', 15]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Cov